<a href="https://colab.research.google.com/github/GabrielaULL/proyecto_ic/blob/main/proyectoIcGabriela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción
**Problema seleccionado:**
Se trata de la resolución del Problema del Viajante (TSP: Traveling Salesperson Problem). En este problema se dispone de un conjunto de ciudades y la tarea consiste en determinar la ruta (un ciclo) de menor distancia que permita visitar cada ciudad exactamente una vez y volver al punto de origen.



#2. Descripción de los algoritmos de resolución
* **Representación de una solución al problema**


La solución se representa mediante una permutación de las ciudades. Cada individuo (solución) es un arreglo en el que cada elemento representa una ciudad (por ejemplo, sus coordenadas o un identificador) y el orden determina la secuencia en la que se visitan las ciudades. Así, una solución ( R ) puede definirse como:

R = [ciudad_1, ciudad_2, ..., ciudad_n]

Esta representación garantiza que, si se cuida el operador de cruce y mutación, se respetará la restricción de visitar cada ciudad solo una vez.

Incialmente se importarán las librerías necesarias, se definirá una instancia de ciudades y sus coordenadas.



In [287]:
# Importar las librerías necesarias
import random
import math
import matplotlib.pyplot as plt

# Definición de las ciudades: nombre y coordenadas (latitud, longitud)
ciudades = {
    "Madrid":    (40.4168, -3.7038),
    "Barcelona": (41.3851,  2.1734),
    "Valencia":  (39.4699, -0.3763),
    "Zamora":    (41.5033, -5.7470)}

# Extraemos la lista de nombres de ciudades.
nombres_ciudades = list(ciudades.keys())

A continuación se encuentran las Funciones para:
* calcular la distancia euclidiana entre dos ciudades ciudades
* obtener la matriz distancias simétricas entre las ciudades.
* Función de distancia total de una ruta (con recorrido del ciclo cerrado)

In [288]:
def distancia(coord1, coord2):
    distancia = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
    return distancia

def calcular_matriz_distancias(ciudades, nombres_ciudades):
    num_ciudades = len(nombres_ciudades)
    matriz = [[0 for _ in range(num_ciudades)] for _ in range(num_ciudades)] #Creando una matriz de dimensiones nxn y llenándola con 0s, donde n es el num_ciudades
    for i in range(num_ciudades):
        for j in range(num_ciudades):
            matriz[i][j] = distancia(ciudades[nombres_ciudades[i]], ciudades[nombres_ciudades[j]]) #Llenando la matriz con las distancias entre ciudades, considerando que las filas corresponden a las n ciudades y las columnas también corresponden a las n ciudades, para establecer todas las parejas
    return matriz

def calcular_distancias_utiles(matriz, nombres_ciudades):
    distancias_utiles = []
    for i in range(len(ciudades)):
        for j in range(i + 1, len(ciudades)):  # Comenzamos desde i+1 para evitar duplicados
            distancia = matriz[i][j]
            distancias_utiles.append((nombres_ciudades[i], nombres_ciudades[j], distancia))
    return distancias_utiles

def distancia_ruta(ruta):
    total = 0
    n = len(ruta)
    for i in range(n):
        total += distancia(ciudades[ruta[i]], ciudades[ruta[(i+1) % n]]) #[(i+1) % n] puede dar como resultado 3 casos: si (i+1)<n el resultado es (i+1), si (i+1)=n el resultado es 0, si (i+1)>n el resultado es la parte entera, pero que en este caso no aplicaría
    return total # Como se ve en el comentario anterior, el total incluye la distancia entre la ultima ciudad (cuando (i+1)=n), donde el índice sería 0.

* **Definición de la función de fitness**

La función de fitness nos ayudará a evaluar la calidad de una solución. En el contexto del TSP, dado que el objetivo es minimizar la distancia, se suele definir el fitness como el inverso de la distancia total, de forma que soluciones con menor recorrido tengan un valor de fitness mayor.
- Solución factible:
Si la ruta cumple todas las restricciones (todas las ciudades sin repeticiones y ciclo cerrado), se define:


In [289]:
def fitness(solucion):
    dist = distancia_ruta(solucion)
    fitness_solucion = 1.0 / dist if dist > 0 else float('inf')
    return fitness_solucion

* **Función de Inicialización**

Se crea una población compuesta por rutas aleatorias.

In [290]:
def crear_poblacion_inicial(tam_poblacion, nombres_ciudades):
    poblacion = []
    #distancia_de_ruta = []
    for _ in range(tam_poblacion): #Se crean nuevas rutas aleatorias y se añaden a la lista de la población el número de veces indicado por el tamaño de la población
        ruta = nombres_ciudades[:] #Se realiza una copia de la lista original para barajar sin afectar la lista original
        random.shuffle(ruta) #Se crea una ruta de forma aleatoria
        poblacion.append(ruta) #Se añade esa ruta a la población

    ruta_y_distancia = {}
    for i, ruta in enumerate(poblacion): # enumerate ya te da el índice y el valor
        total = distancia_ruta(ruta)
        ruta_y_distancia[tuple(ruta)] = total  # Usamos la ruta como clave
    return poblacion, ruta_y_distancia


* **IMPLEMENTACIÓN DE LA BÚSQUEDA ALEATORIA**

Se genera una población inicial mediante permutaciones aleatorias de la lista de ciudades

In [291]:
#BÚSQUEDA ALEATORIA
#Imprimiendo la población inicial de la búsqueda aleatoria
tam_poblacion = 24
poblacion, ruta_y_distancia = crear_poblacion_inicial(tam_poblacion, nombres_ciudades)
# Imprimiendo la distancia de cada ruta:
print("\nRESULTADOS DE BÚSQUEDA ALEATORIA")
print("\nPOBLACIÓN INICIAL Y DISTANCIAS DE RUTAS:")
for i, ruta in enumerate(poblacion):
    total = distancia_ruta(poblacion[i])
    print(f"Ruta {i+1}: {poblacion[i]}, Distancia: {total}")
#Determinando la mejor solución
mejor_solucion = None
mejor_fitness = float('-inf')  # Inicializamos con el peor fitness posible
for solucion in poblacion:
    fitness_solucion = fitness(solucion) #Llamamos a la función fitness enviando como parámetro una solución (correspondiente a cada uno de los elementos de la población generada)
    if fitness_solucion > mejor_fitness:
        mejor_fitness = fitness_solucion
        mejor_solucion = solucion
print("\nMEJOR RUTA ENCONTRADA EN LA BÚSQUEDA ALEATORIA:")
print("\nMejor Ruta Encontrada: ", mejor_solucion)
print("Distancia de la Mejor Ruta: ", (1.0 / mejor_fitness)) # Convertimos el fitness de nuevo a distancia


RESULTADOS DE BÚSQUEDA ALEATORIA

POBLACIÓN INICIAL Y DISTANCIAS DE RUTAS:
Ruta 1: ['Madrid', 'Zamora', 'Valencia', 'Barcelona'], Distancia: 17.202178661695974
Ruta 2: ['Zamora', 'Barcelona', 'Valencia', 'Madrid'], Distancia: 16.8838884801635
Ruta 3: ['Barcelona', 'Zamora', 'Valencia', 'Madrid'], Distancia: 23.080066671202907
Ruta 4: ['Zamora', 'Valencia', 'Madrid', 'Barcelona'], Distancia: 23.080066671202907
Ruta 5: ['Barcelona', 'Madrid', 'Zamora', 'Valencia'], Distancia: 17.202178661695974
Ruta 6: ['Zamora', 'Valencia', 'Madrid', 'Barcelona'], Distancia: 23.080066671202907
Ruta 7: ['Zamora', 'Barcelona', 'Valencia', 'Madrid'], Distancia: 16.8838884801635
Ruta 8: ['Barcelona', 'Valencia', 'Zamora', 'Madrid'], Distancia: 17.202178661695974
Ruta 9: ['Zamora', 'Valencia', 'Barcelona', 'Madrid'], Distancia: 17.202178661695974
Ruta 10: ['Zamora', 'Barcelona', 'Madrid', 'Valencia'], Distancia: 23.080066671202907
Ruta 11: ['Zamora', 'Madrid', 'Barcelona', 'Valencia'], Distancia: 17.2021786

In [292]:
# Probando el código
n=len(ciudades)
# Imprimiendo la lista de ciudades
print("\nLista de ciudades: ",nombres_ciudades)
print("\n")

# Imprimiendo las coordenadas de las ciudades
coordenadas_ciudades = [ciudades[nombre] for nombre in ciudades]
print("\nCoordenadas de las ciudades: ", coordenadas_ciudades)
print("\n")

#Imprimiendo la matriz de distancias entre ciudades
matriz = calcular_matriz_distancias(ciudades, nombres_ciudades)
print("\nMatriz de distancias entre ciudades de ", n, " filas x ", n, " columnas:\n" )
for fila in matriz:
    print(fila)
print("\n")
# Imprimiendo la matriz de distancias, enumerando cada una de las distancias (que serían en total nxn)
print("\nDistancias entre ciudades:\n")
for i in range(len(ciudades)):
    for j in range(len(ciudades)):
        #print(f"Distancia entre ciudad {i+1} y ciudad {j+1}: {matriz[i][j]:.2f}")
        print(f"Distancia entre ciudad {i+1} y ciudad {j+1}: {matriz[i][j]:.2f} ({nombres_ciudades[i]} y {nombres_ciudades[j]}) ")
print("\n")

#Probando el evitar distancias duplicadas y las distancias de 0
"""for i in range(len(ciudades)):
    for j in range(i + 1, len(ciudades)):  # Comenzamos desde i+1 para evitar imprimir duplicados y la distancia de una ciudad a sí misma
        print(f"Distancia entre {nombres_ciudades[i]} y {nombres_ciudades[j]}: {matriz_distancias[i][j]:.2f}")
print("\n")
"""
#Llamando ala funcion calcular_distancias_utiles, pero dejar si es necesario sino eliminarla y descomentar las líneas previas que hacen la impresión sin llamar a una función.
distancias_utiles = calcular_distancias_utiles(matriz, nombres_ciudades)
num_distancias_utiles = int(((n*n)-n)/2)
print("\nDistancias entre ciudades (sin considerar las distancias repetidas ni las distancias entre una ciudad y sí misma)\n")
for i in range(num_distancias_utiles):
  print(distancias_utiles[i])
print("\n")



Lista de ciudades:  ['Madrid', 'Barcelona', 'Valencia', 'Zamora']



Coordenadas de las ciudades:  [(40.4168, -3.7038), (41.3851, 2.1734), (39.4699, -0.3763), (41.5033, -5.747)]



Matriz de distancias entre ciudades de  4  filas x  4  columnas:

[0.0, 5.956432214841364, 3.459606315753282, 2.3141193767824513]
[5.956432214841364, 0.0, 3.188880858545831, 7.921281929081934]
[3.459606315753282, 3.188880858545831, 0.0, 5.742746211526329]
[2.3141193767824513, 7.921281929081934, 5.742746211526329, 0.0]



Distancias entre ciudades:

Distancia entre ciudad 1 y ciudad 1: 0.00 (Madrid y Madrid) 
Distancia entre ciudad 1 y ciudad 2: 5.96 (Madrid y Barcelona) 
Distancia entre ciudad 1 y ciudad 3: 3.46 (Madrid y Valencia) 
Distancia entre ciudad 1 y ciudad 4: 2.31 (Madrid y Zamora) 
Distancia entre ciudad 2 y ciudad 1: 5.96 (Barcelona y Madrid) 
Distancia entre ciudad 2 y ciudad 2: 0.00 (Barcelona y Barcelona) 
Distancia entre ciudad 2 y ciudad 3: 3.19 (Barcelona y Valencia) 
Distancia entre ciuda

* **

* **IMPLEMENTACIÓN DEL ALGORITMO EVOLUTIVO**

  * **Selección de padres mediante torneo**

In [293]:
def select_parents_torneo(poblacion, fitnesses, tam_poblacion):
    parents = []
    ruta_y_distancia_padres={}
    for i in range(2):
        padre1_index = random.randint(0, tam_poblacion-1)
        padre2_index = random.randint(0, tam_poblacion-1)
        if fitnesses[padre1_index]>fitnesses[padre2_index]:
          parents.append(poblacion[padre1_index])
        else:
          parents.append(poblacion[padre2_index])
    return parents

  * **Mutación**
  
  Aplica mutación al individuo intercambiando aleatoriamente dos ciudades.


In [ ]:
def mutacion(solucion, tasa_mutacion=0.1):
    tam = len(solucion)
    for i in range(tam):
        if random.random() < tasa_mutacion:
            j = random.randrange(tam)
            solucion[i], solucion[j] = solucion[j], solucion[i]
    return solucion

  * **Cruce PMX (Partially Mapped Crossover)**

  Se selecciona un segmento aleatorio de padre1 y se establece un mapeo para mantener la validez de la permutación.

In [ ]:
def pmx_crossover(padre1, padre2):
    tam = len(padre1)
    hijo = [None] * tam #Se inicializa la lista con un tamaño establecido con None en todas sus posiciones
    inicio, fin = sorted([random.randint(0, tam - 1) for _ in range(2)]) #Se selecciona dos puntos de cruce aleatorios
    hijo[inicio:fin+1] = padre1[inicio:fin+1] #Se copia el segmento de padre1 en el hijo
    mapeo = {padre1[i]: padre2[i] for i in range(inicio, fin+1)}  #Se mapea, es decir que se arman correspondencias entre padre1 y padre2 en cada una de las posiciones desde el inicio hasta el fin
    for i in range(tam): #Completar el hijo con valores de padre2, evitando repeticiones
        if hijo[i] is None: #El código ejecuta el siguiente bloque si la posición i del hijo está vacía
            valor = padre2[i] #Se copia la posición i del padre2 en la variable valor
            while valor in hijo: #Se entra en el siguiente bucle cuando el contenido de la variable valor ya está en el hijo, es decir cuando la ciudad ya forma parte de la nueva ruta que constituye el hijo
                valor = mapeo.get(valor, valor) #Se obtiene del mapeo previo, la correspondencia y se copia ese valor en la variable valor
            hijo[i] = valor #Se copia el contenido de la variable valor en la posición i del hijo
    return hijo

  * **Algoritmo Evolutivo**

  LA MUTACIÓN Y EL CRUCE DEBERÍAN FUNCIONAR SIN PROBLEMA, PROBAR EL SIGUIENTE ALGORITMO EVOLUTIVO. EL PUNTO 9 Y EL MAIN QUE ESTÁN EN COPILOT

In [1]:
def algoritmo_evolutivo(lista_ciudades, tam_poblacion, generaciones, tasa_mutacion):
    poblacion = crear_poblacion_inicial(tam_poblacion, lista_ciudades)
    mejor_ruta = min(poblacion, key=lambda ruta: distancia_ruta(ruta))
    mejor_distancia = distancia_ruta(mejor_ruta)
    for gen in range(generaciones):
        nueva_poblacion = []
        for _ in range(tam_poblacion):
            padre1 = seleccion_torneo(poblacion)
            padre2 = seleccion_torneo(poblacion)
            hijo = cruzamiento(padre1, padre2)
            hijo = mutacion(hijo, tasa_mutacion)
            nueva_poblacion.append(hijo)
        poblacion = nueva_poblacion
        candidato = min(poblacion, key=lambda ruta: distancia_ruta(ruta))
        dist_candidato = distancia_ruta(candidato)
        if dist_candidato < mejor_distancia:
            mejor_ruta = candidato
            mejor_distancia = dist_candidato
        print(f"Generación {gen+1}: Mejor distancia = {mejor_distancia:.4f}")

    return mejor_ruta, mejor_distancia


#3. Experimentación

In [295]:
fitnesses = [fitness(solution) for solution in poblacion]
parents = select_parents_torneo(poblacion, fitnesses, tam_poblacion)
print("\nPADRES SELECCIONADOS ")
for i in range(len(parents)):
    total = distancia_ruta(parents[i])
    print(f"Padre {i}: con ruta {parents[i]} y Distancia: {total}")


PADRES SELECCIONADOS 
Padre 0: con ruta ['Madrid', 'Barcelona', 'Valencia', 'Zamora'] y Distancia: 17.202178661695974
Padre 1: con ruta ['Zamora', 'Barcelona', 'Valencia', 'Madrid'] y Distancia: 16.8838884801635
